<a href="https://colab.research.google.com/github/giannismantzaris-cmd/DAMA60/blob/main/Topic5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/giannismantzaris-cmd/DAMA60.git
!python DAMA60/WA2/Topic5.py

fatal: destination path 'DAMA60' already exists and is not an empty directory.
=== Topic 5: Flajolet–Martin on a Wikipedia page ===
Enter Wikipedia URL (or press Enter for default 'Data science wikipedia page'): Traceback (most recent call last):
object address  : 0x7842e2431d20
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [ ]:
#DAMA60 HW2 Topic5
#Topic 5 – Python
#Flajolet–Martin distinct word estimation from a Wikipedia page.

#How to run (IDLE or terminal):
#- Just run this file. You will be prompted for a Wikipedia URL.
#  Example: https://en.wikipedia.org/wiki/Data_science
#- If you press Enter, a default URL will be used.

#No third-party libraries required.

#Import needed libraries
import re
import sys
import html
import urllib.request
import urllib.error
import statistics
from typing import List, Tuple

# ---------------------------- Text fetching & cleaning ----------------------------

def fetch_text(url: str):
    #Fetch raw HTML from URL and return a best-effort cleaned plain text.
    try:
        headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) "
                                 "AppleWebKit/537.36 (KHTML, like Gecko) "
                                 "Chrome/120.0 Safari/537.36"}
        req = urllib.request.Request(url, headers=headers)
        with urllib.request.urlopen(req, timeout=20) as resp:
            charset = resp.headers.get_content_charset() or "utf-8"
            html_bytes = resp.read()
    except urllib.error.URLError as e:
        print(f"[Error] Failed to fetch URL: {e}")
        return ""

    page = html_bytes.decode(charset, errors="replace")# Decode the raw HTML bytes into a string using the detected character set (e.g., UTF-8).
                                                        # If some characters cannot be decoded, they are replaced with a safe placeholder.
    page = html.unescape(page) # Convert HTML entities (like &amp;, &lt;, &gt;) into their actual characters (&, <, >).
    page = re.sub(r"(?is)<script.*?>.*?</script>", " ", page) # Remove <script>...</script> blocks (JavaScript code).
                                                                # (?is) makes the regex case-insensitive and allows to match newlines.
    page = re.sub(r"(?is)<style.*?>.*?</style>", " ", page) # Remove <style>...</style> blocks (CSS definitions).
    page = re.sub(r"(?is)<!--.*?-->", " ", page) # Remove HTML comments <!-- ... -->.
    text = re.sub(r"(?s)<[^>]+>", " ", page) # Remove any remaining HTML tags like <div>, <p>, <a>, etc.
                                                # (?s) makes . match newlines too.
    text = re.sub(r"\s+", " ", text).strip() # Collapse multiple spaces, tabs, or newlines into a single space.
                                                # Also strip leading and trailing whitespace.
    return text

#Tokenization
def tokenize_words(text: str): # The function will return a list of strings (tokens).
    text = text.lower() # Lowercase the text so that "Data" and "data" are treated as the same word
    for ch in text:
        if not (ch.isalpha() or ch in "'-"): #Checks if the character is a letter, apostrophe, or hyphen
            text = text.replace(ch, " ")# Replace any character that is NOT a letter (a–z, A–Z), apostrophe ('), or hyphen (-)
                                            # with a space. This removes digits, punctuation, symbols, etc.
    tokens = [re.sub(r"(^[-']+|[-']+$)", "", t) for t in text.split()] # Split into tokens (words) and remove leading/trailing apostrophes or hyphens
                                                                        # Example: "'hello-'" → "hello"
    cleaned = []
    for t in tokens:
       if t:                # keep only non-empty
        cleaned.append(t)
    return cleaned # Return the cleaned list of tokens, excluding any empty strings

#Custom hash function
def myHash(text: str):
    hash = 0
    for ch in text:
        hash = (hash * 281 ^ ord(ch) * 997) & 0xFFFFFFFF #281 and 997 are prime numbers
    return hash

# ---------------------------- Flajolet–Martin ----------------------------

def trailing_zeros(n: int):
    #Count trailing zero bits in n (for n > 0). If n == 0, return 64.
    if n == 0:
        return 64
    tz = 0
    while n & 1 == 0: # while least significant bit is 0
        tz = tz + 1
        n = n >> 1 # shift right (divide by 2), move to next bit
    return tz

def fm_estimate(words: List[str], hash_params: List[Tuple[int, int]], m_bits: int = 32): # hash_params: List[Tuple[int, int]]: A list of tuples, each containing two integers (a, b).
                                                                                        # Each (a, b) pair defines one hash function of the form h(x) = (a * x + b) mod M.
                                                                                        # m_bits: Sets the number of bits in the hash range (M = 2^32).
    #Run Flajolet–Martin with multiple (a,b) linear hashes.
    # This function takes a stream of words, applies multiple hash functions, and estimates the number of distinct elements using the Flajolet–Martin algorithm
    M = 2** m_bits #Compute M = 2 ** m_bits
    max_tz_per_hash = [0] * len(hash_params) #Initialize a list of zeros, one entry per hash function, to store the maximum trailing zeros

    for w in words: #look at each word from the text one by one

        word_hash = myHash(w) # Convert the word into an integer using the custom hash function myHash defined above
        for i, (a, b) in enumerate(hash_params):
            hv = (a*word_hash+b) % M   # Compute hash value in range [0, M)
            r = trailing_zeros(hv)# Count trailing zeros in binary form of hv

            if r > max_tz_per_hash[i]: # Update max trailing zeros for this hash function
                max_tz_per_hash[i] = r

    estimates = [2 ** r for r in max_tz_per_hash] # Convert max trailing zeros into distinct-count estimates
    combined = statistics.median(estimates)  # Compute median using python library statistics

    return max_tz_per_hash, estimates, combined

# ---------------------------- Main program ----------------------------

print("=== Topic 5: Flajolet–Martin on a Wikipedia page ===")
url = input("Enter Wikipedia URL (or press Enter for default 'Data science wikipedia page'): ").strip()
if not url:
    url = "https://en.wikipedia.org/wiki/Data_science"

print(f"[1/4] Fetching page: {url}")
text = fetch_text(url)
if not text:
    print("No text fetched. Exiting.")
    sys.exit(1)

print("[2/4] Tokenizing words...")
words = tokenize_words(text) #cleans the page text and returns a list of words (tokens).
total_tokens = len(words) #total number of tokens in the text (including duplicates).
unique_tokens = len(set(words)) # set(words): removes duplicates, leaving only distinct words.
                                #len(set(words)): the exact number of unique words.
print(f"Tokens: {total_tokens:,} | Unique (exact set): {unique_tokens:,}")

# Independent hash functions (different (a,b) pairs), using hexadecimal constants
# Each pair represents the coefficients of a simple linear hash function
print("[3/4] Running Flajolet–Martin...")

# The constants a and b are written in hexadecimal for compactness.
#They are just big integers used to define different hash functions.
#Hex is common in hashing
hash_params = [             # The values are written in hexadecimal (like 0x27d4eb2f), but they’re just big integers.
    (0x27d4eb2f, 0x165667b1),
    (0x9e3779b1, 0x85ebca6b),
    (0xc2b2ae35, 0x27d4eb2f),
    (0x165667b1, 0x9e3779b1),
    (0x85ebca6b, 0xc2b2ae35),
    (0x27d4eb2f, 0x85ebca6b),
    (0x9e3779b1, 0x27d4eb2f),
    (0xc2b2ae35, 0x165667b1),
    (0x2545f491, 0x9e3779b1),
    (0x94d049bb, 0xc2b2ae35),
    (0x369dea0f, 0x94d049bb),
    (0x7f4a7c15, 0x2545f491),
]
max_tz, ests, combined = fm_estimate(words, hash_params, m_bits=32)

print("[4/4] Results")
print("Hash # | max trailing zeros (R) | estimate (2^R)")
i = 1
for index in range(len(max_tz)):
    r = max_tz[index]
    e = ests[index]
    print(f"Hash {i:<2} | R = {r:<3} | Estimate = {e:<8}")
    i += 1
print(f"\nCombined Flajolet–Martin estimate (median): {combined:,.2f} distinct words")
print(f"Exact unique tokens(set)             : {unique_tokens:,}")


=== Topic 5: Flajolet–Martin on a Wikipedia page ===
